## 测新 - 脚本       

In [1]:
import web
import logging
import logging.config
import pymysql
import yaml

import pandas as pd
import numpy as np

from config import *
from media_toutiao import TTCreatePlan  # 末日
from media_toutiao import TTCreatePlanDg  # 帝国
from media_toutiao import TTCreatePlanDdj  # 大东家
from media_gdt import GDTCreatePlan  # 末日
from media_gdt import GDTCreatePlanDg  # 帝国
from media_gdt import GDTCreatePlanSite  # 末日版位

from ptom_model import PtomPredictNew

In [2]:
def get_plan_online(plan_result):
    """[头条]

    Args:
        plan_result ([dataframe]): [过滤后的新建计划集]]

    Returns:
        [json]: []
    """
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
    open_api_url_prefix = "https://ptom.caohua.com/"
    # open_api_url_prefix = "https://ptom-pre.caohua.com/"   ## 预发布环境
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 10
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print("[INFO]：结束！")
    return rsp_data

In [3]:
def get_imageid_tp(image_ids):
    """ 素材名：精准 """
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)
    
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name,
            label_ids
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
    '''
    cur.execute(sql.format(image_ids))
    result = pd.read_sql(sql.format(image_ids), conn)
    # 关闭链接
    cur.close()
    conn.close()
    return result

## 新建计划
> 末日：CX = TTCreatePlan()          
> 大东家：CX = TTCreatePlanDdj()  

In [4]:
CX = TTCreatePlan()

候选计划deep_bid_type分布： BID_PER_ACTION    36
Name: deep_bid_type, dtype: int64
候选计划smart_bid_type分布： SMART_BID_NO_BID    19
SMART_BID_CUSTOM    17
Name: smart_bid_type, dtype: int64
候选计划inventory_type分布： [INVENTORY_UNION_SLOT, INVENTORY_AWEME_FEED, INVENTORY_FEED, INVENTORY_UNION_SPLASH_SLOT, INVENTORY_VIDEO_FEED, INVENTORY_HOTSOON_FEED, INVENTORY_TOMATO_NOVEL]    26
[INVENTORY_UNION_SLOT]                                                                                                                                              6
[INVENTORY_UNION_SLOT, INVENTORY_AWEME_FEED, INVENTORY_FEED, INVENTORY_TOMATO_NOVEL]                                                                                2
[INVENTORY_UNION_SLOT, INVENTORY_AWEME_FEED, INVENTORY_FEED]                                                                                                        1
[INVENTORY_UNION_SLOT, INVENTORY_AWEME_FEED, INVENTORY_FEED, INVENTORY_HOTSOON_FEED, INVENTORY_VIDEO_FEED, INVENTORY_TOMATO_NOVEL]     

In [5]:
plan_result = CX.Process()

未建计划素材量: 3
3000返新素材量: 8
[INFO]：新素材提取完毕！
训练数据游戏包分布： 1001703    1615
1001772    1128
1001447      70
1002028      60
Name: game_id, dtype: int64
训练数据标签分布：  0    2557
 1     212
-1     104
Name: label, dtype: int64


0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=15 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=15 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Info] Number of positive: 151, number of negative: 1787
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004178 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6887
[LightGBM] [Info] Number of data points in the train set: 1938, number of used features: 154
[LightGBM] [Warning] min_data_in_leaf is set=101, min_child_samples=15 will be ignored. Current value: min_data_in_leaf=101
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.077915 -> initscore=-2.471014
[LightGBM] [Info] Start training from score -2.471014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's auc: 0.617784	valid_1's auc: 0.525601
Traini

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[94]	training's auc: 0.794498	valid_1's auc: 0.56303
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[95]	training's auc: 0.79452	valid_1's auc: 0.564073
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[96]	training's auc: 0.795895	valid_1's auc: 0.563626
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[97]	training's auc: 0.795751	valid_1's auc: 0.564222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[98]	training's auc: 0.797	valid_1's auc: 0.563455
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[99]	training's auc: 0.796944	valid_1's auc: 0.563966
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	training's auc: 0.797085	valid_1's auc: 0.564243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[101]	training's auc: 0.79

In [6]:
print(plan_result.shape)
plan_result[['ad_account_id','game_id','image_id','deep_bid_type','cpa_bid','smart_bid_type','inventory_type','title_list']]

(5, 42)


,ad_account_id,game_id,image_id,deep_bid_type,cpa_bid,smart_bid_type,inventory_type,title_list
0,11600,1001703,46542,BID_PER_ACTION,683.73,SMART_BID_CUSTOM,[],[{'title': '智商149卡在了39关，这款游戏太烧脑，你能过几关？'}]
1,11599,1001703,46561,BID_PER_ACTION,1236.45,SMART_BID_NO_BID,[],"[{'title': '闯关+逃生，没WiFi也能玩一天，试试你能过几关？'}, {'tit..."
2,11598,1001703,46478,BID_PER_ACTION,839.81,SMART_BID_CUSTOM,[],"[{'title': '这款丧尸游戏注册就送ssr英雄，升级送十连抽！', 'creativ..."
30,11597,1001703,46477,BID_PER_ACTION,360.88,SMART_BID_NO_BID,[],"[{'title': '智商149卡在了39关，这款游戏太烧脑，你能过几关？', 'crea..."
59,11596,1001703,46560,BID_PER_ACTION,360.88,SMART_BID_NO_BID,[],"[{'title': '智商149卡在了39关，这款游戏太烧脑，你能过几关？', 'crea..."


In [14]:
# import json
# import requests
# get_plan_online(plan_result)

[INFO]：结束！


{'code': 200, 'data': True, 'msg': ''}

In [8]:
get_imageid_tp(plan_result['image_id'])

,image_id,image_name,label_ids
0,46477,SSR5400-竖版（2D外部混剪）ZM,"301,277"
1,46478,SSR5401-竖版（2D外部混剪）ZM,"277,301"
2,46542,SSR5407-横版（3D混剪）JQ,20
3,46560,SSR5410-横版（3D混剪）RKQ,"20,235"
4,46561,SSR5409-横版（3D混剪）RKQ,"277,301"


## 复制计划
> CX = PtomPredictNew()        

In [9]:
PCP = PtomPredictNew()

In [10]:
action = 'plan_pay_copy'
media = '头条'
game = '末日'

In [11]:
# copy_result, ret = PCP.Process(action, media, game)

In [12]:
print(copy_result.shape)
copy_result[['ad_account_id','game_id','image_id','deep_bid_type','cpa_bid','smart_bid_type','inventory_type']]

NameError: name 'copy_result' is not defined

In [ ]:
# import json
# import requests
# get_plan_online(copy_result)

In [ ]:
get_imageid_tp(copy_result['image_id'])